In [ ]:
from pyfeti.src.utils import DomainCreator, dict2dfmap, create_selection_operator, DiskCreator, DofManager
from pyfeti.src.feti_solver import SerialFETIsolver, SolverManager, cyclic_eig
from pyfeti.src.linalg import ProjLinearSys
from pyfeti.src.cyclic import Cyclic_Constraint
from pyfeti.cases.case_generator import FETIcase_builder
from scipy import sparse
import amfe
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
r_in_1 = 6
r_out_1 = 18
angle=45
theta_0=(90-angle/2)
angle_divisions = 21
radial_divisions = 12
dobj1 = DiskCreator(r_in=r_in_1,r_out=r_out_1, sector_angle=angle, theta_0=theta_0,angle_divisions=angle_divisions,radial_divisions=radial_divisions)


mesh_file1 = 'mesh_disk.msh'
dobj1.save_gmsh_file(mesh_file1)

m1 = amfe.Mesh()
m1.import_msh(mesh_file1)



fig, ax1 = plt.subplots(1,1,figsize=(15,15))
amfe.plot2Dmesh(m1,ax=ax1)


mult=1.2
for ax in (ax1,):
    ax.set_xlim([-mult*r_out_1,mult*r_out_1])
    ax.set_ylim([-mult*r_out_1,mult*r_out_1])
    ax.set_aspect('equal')
    ax.set_xlabel('Width [m]')
    ax.set_ylabel('Heigh [m]')
plt.legend('off')

In [ ]:
m_list = [m1.rot_z(i*angle) for i in range(int(360/angle))]
fig1, ax4 = plt.subplots(1,1,figsize=(15,15))
for m in m_list:
    ax4 = amfe.plot2Dmesh(m,ax=ax4)

mult=1.2
ax4.set_xlim([-mult*r_out_1,mult*r_out_1])
ax4.set_ylim([-mult*r_out_1,mult*r_out_1])
ax4.set_aspect('equal')
ax4.set_xlabel('Width [m]')
ax4.set_ylabel('Heigh [m]')
plt.legend('off')

In [ ]:
# creating material
my_material = amfe.KirchhoffMaterial(E=210.0E9, nu=0.3, rho=7.86E3, plane_stress=True, thickness=1.0)

my_system1 = amfe.MechanicalSystem()
my_system1.set_mesh_obj(m1)
my_system1.set_domain(3,my_material)



K1, _ = my_system1.assembly_class.assemble_k_and_f()
M1 = my_system1.assembly_class.assemble_m()


In [ ]:
val, V = np.linalg.eig(K1.A)
val

In [ ]:
try:
    connectivity = []
    for _,item in m1.el_df.iloc[:, m1.node_idx:].iterrows():
        connectivity.append(list(item.dropna().astype(dtype='int64')))
    m1.el_df['connectivity'] = connectivity
except:
    pass
    
id_matrix = my_system1.assembly_class.id_matrix
id_map_df = dict2dfmap(id_matrix)
el_df = m1.el_df
nodes_coord = m1.nodes
dirichlet_label = 4
#cyclic_left_label = 2
#cyclic_right_label = 1
cyclic_left_label = 1
cyclic_right_label = 2
sector_angle = angle
unit='deg'
tol_radius = 1.0e-3
dimension=2
cyc_obj = Cyclic_Constraint(id_map_df,
                            el_df,
                            nodes_coord,
                            dirichlet_label,
                            cyclic_left_label,
                            cyclic_right_label,
                            sector_angle,
                            unit=unit,
                            tol_radius = 1.0e-3,
                            dimension=2)

n = 0
Cn = cyc_obj.build_complex_contraint(n)
B11 = cyc_obj.s.build_B('d')
B11 = B11 + 1.0J*0*B11

B = sparse.vstack([Cn,B11]).A
#B = B11.A

BBT_inv = np.linalg.inv(B.dot(B.T))
P = np.eye(M1.shape[0]) - B.T.dot(BBT_inv.dot(B))
P = P

In [ ]:
dtype = np.complex
K_dict = {} 
K_dict[1] = K1 + + 1J*sparse.csr_matrix(K1.shape)

M_dict = {} 
M_dict[1] = M1 + + 1J*sparse.csr_matrix(M1.shape)


B_dict = {}
B_dict = { 1 : {(1,1) : B}}

if dtype == np.float:
    mult = 1
else:
    mult = 2
v = 100*np.random.rand(mult*K1.shape[0])
v.dtype = dtype 
f_dict = {1 : v}



In [ ]:
lo_obj = ProjLinearSys(K1,M1,P)
Dp_new = lo_obj.getLinearOperator()


u0 = P.dot(v)  
eigval_, V_wp_ = sparse.linalg.eigsh(Dp_new ,k=9,v0=u0)
val_wp_ = np.sort(1/eigval_)
freq = np.sqrt(val_wp_)/(2.0*np.pi)
freq

In [ ]:
Cn.dot(u0)

In [ ]:
B11.dot(u0)

In [ ]:
fake_feti = SerialFETIsolver(K_dict,B_dict,f_dict,dtype=dtype,tolerance=1.0e-6)
manager = fake_feti.manager
manager.build_local_to_global_mapping()


def system(u,tol=1.0e-12):
    f = P.T.dot(M1.dot(P.dot(u)))
    f_dict = manager.vector2localdict(f,manager.global2local_primal_dofs)
    feti_obj = SerialFETIsolver(K_dict,B_dict,f_dict,tolerance=tol,dtype=dtype,max_int=500)
    solution_obj = feti_obj.solve()
    u_dict = solution_obj.u_dict
    return solution_obj.displacement
    
un = system(u0)
u0 = P.dot(v)  

In [ ]:
B.dot(u0)

In [ ]:
system(u0)
un = system(u0)

In [ ]:
D_wp = sparse.linalg.LinearOperator(shape=M1.shape, matvec = lambda x : system(x), dtype=dtype)

In [ ]:
eigval_without_projection_, V_wp_ = sparse.linalg.eigs(D_wp,k=9,v0=u0)


In [ ]:
val_wp_ = np.sort(1/-eigval_without_projection_)
freq_dual_wp_ = np.real(np.sqrt(val_wp_)/(2.0*np.pi))
freq_dual_wp_

In [ ]:
eigval_without_projection_

In [ ]:

u1=V_wp_
my_system1.u_output = list(u1.T)


In [ ]:
B.dot(u1[:,0])

In [ ]:
p0 = 10.0
fig, ax1_list = plt.subplots(3,3,figsize=(15,15))
counter = 0
offset = 8
delta_ = 1.0
mult = 1.1
for ax_ij in ax1_list:
    for ax in ax_ij:
        amfe.plot_2D_system_solution(my_system1,u_id=(counter),ax=ax,factor=p0)
        ax.set_xlim([-mult*r_out_1,mult*r_out_1])
        ax.set_ylim([-mult*r_out_1,mult*r_out_1])
        ax.set_aspect('equal')
        ax.set_xlabel('Width [m]')
        ax.set_ylabel('Heigh [m]')
        counter+=1
plt.tight_layout()
plt.legend('off')

In [ ]:
from amfe.cyclic.cyclic import create_voigt_rotation_matrix, create_rotated_component


my_comp = amfe.MechanicalSystem()
my_comp.set_mesh_obj(m1)
my_comp.set_domain(3,my_material)
my_comp.u_output = list(u1.T)

sys_list = []
v1 = u1.copy()
for i,m in enumerate(m_list):
    my_system1 = amfe.MechanicalSystem()
    my_system1.set_mesh_obj(m)
    my_system1.set_domain(3,my_material)
    theta = np.deg2rad(i*angle)
    R = create_voigt_rotation_matrix(u1.shape[0],theta,unit='rad')
    ej_n_theta = np.exp(-1J*i*n*theta)     
    v = R.dot(ej_n_theta*v1)
    my_system1.u_output = list(v.T)
    #comp = create_rotated_component(my_comp,cyc_obj.s,sector_id=i, node_id=n, theta=theta, compute_left=True, unit='rad', dimension=2)
    sys_list.append(my_system1)
    
    
p0 = 10.0
fig, ax1_list = plt.subplots(3,3,figsize=(15,15))
counter = 0
offset = 8
delta_ = 1.0
mult = 1.1
for ax_ij in ax1_list:
    for ax in ax_ij:
        for sys in sys_list:
            amfe.plot_2D_system_solution(sys,u_id=(counter),ax=ax,factor=p0)
        ax.set_xlim([-mult*r_out_1,mult*r_out_1])
        ax.set_ylim([-mult*r_out_1,mult*r_out_1])
        ax.set_aspect('equal')
        ax.set_xlabel('Width [m]')
        ax.set_ylabel('Heigh [m]')
        counter+=1
plt.tight_layout()
plt.legend('off')

In [ ]:
n

In [ ]:
r_in_1 = 6
r_out_1 = 12
r_in_2 = r_out_1
r_out_2 = 18
angle=45
theta_0=(90-angle/2)
angle_divisions = 3
radial_divisions = 3
dobj1 = DiskCreator(r_in=r_in_1,r_out=r_out_1, sector_angle=angle, theta_0=theta_0,angle_divisions=angle_divisions,radial_divisions=radial_divisions)
dobj2 = DiskCreator(r_in=r_in_2,r_out=r_out_2, sector_angle=angle, theta_0=theta_0,angle_divisions=angle_divisions,radial_divisions=radial_divisions)

mesh_file1 = 'mesh_1.msh'
mesh_file2 = 'mesh_2.msh'
dobj1.save_gmsh_file(mesh_file1)
dobj2.save_gmsh_file(mesh_file2)

m1 = amfe.Mesh()
m1.import_msh(mesh_file1)

m2 = amfe.Mesh()
m2.import_msh(mesh_file2)

fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(15,15))
amfe.plot2Dmesh(m1,ax=ax1)
amfe.plot2Dmesh(m2,ax=ax1)
amfe.plot2Dmesh(m1,ax=ax2)
amfe.plot2Dmesh(m2,ax=ax3)

mult=1.2
for ax in (ax1,ax2,ax3,ax4):
    ax.set_xlim([-mult*r_out_2,mult*r_out_2])
    ax.set_ylim([-mult*r_out_2,mult*r_out_2])
    ax.set_aspect('equal')
    ax.set_xlabel('Width [m]')
    ax.set_ylabel('Heigh [m]')
plt.legend('off')

In [ ]:
m1.el_df

In [ ]:
m1.connectivity

In [ ]:
list(m1.el_df['phys_group'])

In [ ]:
my_dict = {}
my_dict['idx_gmsh'] = list(range(1,17))
my_dict['phys_group'] = [6, 7, 8, 9, 4, 4, 2, 2, 5, 5, 1, 1, 3, 3, 3, 3]
my_dict['connectivity'] = [[0],
                                   [2],
                                   [6],
                                   [8],
                                   [0,1],
                                   [1,2],
                                   [2,5],
                                   [5,8],
                                   [8,7],
                                   [7,6],
                                   [6,3],
                                   [3,0],
                                   [0,1,4,3],
                                   [1,2,5,4],
                                   [3,4,7,6],
                                   [4,5,8,7]]

In [ ]:
import pandas as pd

pd.DataFrame(my_dict)


In [ ]:
# creating material
my_material = amfe.KirchhoffMaterial(E=210.0E9, nu=0.3, rho=7.86E3, plane_stress=True, thickness=1.0)

my_system1 = amfe.MechanicalSystem()
my_system1.set_mesh_obj(m1)
my_system1.set_domain(3,my_material)

In [ ]:
my_system1.assembly_class.id_matrix

In [ ]:
id_map_df = np.array(np.array(list(range(18))))

In [ ]:
l = id_map_df.reshape(9,2)

In [ ]:
l

In [ ]:
pd.DataFrame(l)

In [ ]:
m1.nodes

In [ ]:
plt.plot(m1.nodes[:,0].T,m1.nodes[:,1].T,'o')

In [ ]:
l = np.array(my_dict['phys_group'] )

In [ ]:
l[[1,0,0,1]]

In [ ]:
l

In [ ]:
l[[True,True,False]]

In [ ]:
np.zerosl = [1,1,4,10]

In [ ]:
l[(True,True,False,False)]

In [ ]:
K1.dtype

type(K1)

In [ ]:
K1.dot(v)

K4 = K1.copy() + 1J*sparse.csr_matrix(K1.shape)

In [ ]:
K4.A

In [ ]:
K1.shape

In [ ]:
K4.shape

In [ ]:
lu = sparse.linalg.splu(K4)

In [ ]:
lu.solve(u0)

In [ ]:
v0


In [ ]:
u0